In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [12]:


standings_url = 'https://fbref.com/en/comps/12/2021-2022/2021-2022-La-Liga-Stats'

data = requests.get(standings_url)

soup = BeautifulSoup(data.text)
standings_table= soup.select('table.stats_table')[0]
links = standings_table.find_all('a')

links = [l.get('href') for l in links]

links = [l for l in links if '/squads/' in l ]

team_urls = [f'https://fbref.com{l}' for l in links]
team_url = team_urls[0]
data= requests.get(team_url)
import pandas as pd

matches = pd.read_html(data.text, match='Scores & Fixtures')
soup = BeautifulSoup(data.text)
links = soup.find_all('a')

links = [l.get('href') for l in links]
links = [l for l in links if l and 'all_comps/shooting/' in l]

data = requests.get(f'https://fbref.com{links[0]}')
shooting = pd.read_html(data.text,match='Shooting')[0]
shooting.columns = shooting.columns.droplevel()
shooting.columns
team_data = matches[0].merge(shooting[['Date', 'Sh', 'SoT', 'Dist', 'FK',
       'PK', 'PKatt', 'xG'
       ]],on='Date')


In [13]:
team_data

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Referee,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt,xG_y
0,2021-08-14,22:00,La Liga,Matchweek 1,Sat,Away,W,4,1,Alavés,...,César Soto,Match Report,NaN,19,7,18.7,2.0,0,0,1.7
1,2021-08-22,22:00,La Liga,Matchweek 2,Sun,Away,D,3,3,Levante,...,Guillermo Cuadra,Match Report,NaN,20,5,20.4,1.0,0,0,1.6
2,2021-08-28,22:00,La Liga,Matchweek 3,Sat,Away,W,1,0,Betis,...,Alejandro Hernández,Match Report,NaN,13,3,18.3,0.0,0,0,0.9
3,2021-09-12,21:00,La Liga,Matchweek 4,Sun,Home,W,5,2,Celta Vigo,...,José Sánchez,Match Report,NaN,14,10,12.8,0.0,1,1,3.2
4,2021-09-15,21:00,Champions Lg,Group stage,Wed,Away,W,1,0,it Inter,...,Daniel Siebert,Match Report,NaN,12,2,16.3,0.0,0,0,1.2
5,2021-09-19,21:00,La Liga,Matchweek 5,Sun,Away,W,2,1,Valencia,...,Pablo González,Match Report,NaN,18,10,18.4,0.0,0,0,1.3
6,2021-09-22,22:00,La Liga,Matchweek 6,Wed,Home,W,6,1,Mallorca,...,Alberola Rojas,Match Report,NaN,18,11,15.2,0.0,0,0,2.6
7,2021-09-25,21:00,La Liga,Matchweek 7,Sat,Home,D,0,0,Villarreal,...,Jesús Gil,Match Report,NaN,15,2,19.2,0.0,0,0,1.1
8,2021-09-28,21:00,Champions Lg,Group stage,Tue,Home,L,1,2,md Sheriff Tiraspol,...,Lawrence Visser,Match Report,NaN,30,10,18.0,2.0,1,1,3.1
9,2021-10-03,16:15,La Liga,Matchweek 8,Sun,Away,L,1,2,Espanyol,...,Guillermo Cuadra,Match Report,NaN,18,4,18.1,2.0,0,0,1.6


In [11]:
standings_url = 'https://fbref.com/en/comps/12/2021-2022/2021-2022-La-Liga-Stats'
years = list(range(2022, 2020, -1))
all_matches = []
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]

        links_shooting = [l for l in links if l and 'all_comps/shooting/' in l]
        data_shooting = requests.get(f"https://fbref.com{links[0]}")

        links_passing = [l for l in links if l and 'all_comps/passing/' in l]
        data_passing = requests.get(f"https://fbref.com{links[0]}")

        links_defensive = [l for l in links if l and 'all_comps/defense/' in l]
        data_defensive = requests.get(f"https://fbref.com{links[0]}")

        shooting = pd.read_html(data_shooting.text, match="Shooting")[0]
        passing = pd.read_html(data_passing.text, match="Passing")[0]
        defensive = pd.read_html(data_defensive.text, match="Defensive Actions")[0]

        shooting.columns = shooting.columns.droplevel()
        passing.columns = passing.columns.droplevel()
        defensive.columns = defensive.columns.droplevel()

        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt",'xG']], on="Date")
        except ValueError:
            continue
       

        try:
            team_data = matches.merge(passing[["Date",'Cmp','Att','TotDist','PrgDist','Ast','xAG', 'xA','KP']], on="Date")
        except ValueError:
            continue
        
		
        try:
            team_data = matches.merge(defensive[["Date",'Tkl',	'TklW'	,'Def 3rd'	,'Mid 3rd',	'Att 3rd','Tkl'	,'Att',	'Tkl%'	,'Lost'	,'Blocks',	'Sh',	'Pass'	,'Int'	,'Tkl+Int'	,'Clr',	'Err' ]], on="Date")
        except ValueError:
            continue


        team_data = team_data[team_data["Comp"] == "La Liga"]       
        
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(5)

   

   

ConnectionError: HTTPSConnectionPool(host='fbref.comhttps', port=443): Max retries exceeded with url: //fbref.com/pt/squads/53a2f082/2021-2022/Real-Madrid-Stats (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000026142FA9240>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [ ]:
match_df = pd.concat(all_matches)
match_df.columns = [c.lower() for c in match_df.columns]
match_df.to_csv('matches.csv')